In [27]:
import numpy as np
import pandas as pd
import numba as nb
from typing import List
import math
from fractions import Fraction

from numpy import ndarray

np.seterr(divide='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [28]:
U = {
    (4, 'Маг'): {
        'Z': 5,
        'D': 3,
        'K': 4,
    },
    (5, 'Рыцарь'): {
        'Z': 4,
        'D': 5,
        'K': 7,
    },
    (40, 'Дракон'): {
        'Z': 50,
        'D': 35,
        'K': 45,
    }
}

R = {
    'Z': 450,
    'D': 300,
    'K': 500,
}

# Тупой перебор

In [29]:
s = np.array([u[0] for u in U]).astype('float64')
p = np.array([[Uvv for Uvk, Uvv in Uv.items()] for Uk, Uv in U.items()]).astype('float64')
r = np.array([Rv for Rk, Rv in R.items()]).astype('float64')

print(s)
print(p)
print(r)

[ 4.  5. 40.]
[[ 5.  3.  4.]
 [ 4.  5.  7.]
 [50. 35. 45.]]
[450. 300. 500.]


In [30]:
@nb.njit('float64[:](float64[:], float64[:,:], float64[:])', parallel=True)
def dump(powers, prices, resources):
    best = 0
    best_x = np.zeros_like(powers)

    # maxes = np.floor(np.min(1 / np.divide(prices, resources.T), axis=1))
    maxes = np.floor(np.array([np.min(r) for r in 1 / np.divide(prices, resources.T)]))

    for X1 in range(0, int(maxes[0])):
        for X2 in range(0, int(maxes[1])):
            # print(resources - (prices[0] * X1 + prices[1] * X2), prices[2])
            x = np.array(
                [X1, X2, np.floor(np.min(np.divide(resources - (prices[0] * X1 + prices[1] * X2), prices[2])))])
            if np.any(x < 0):
                break
            strength = np.sum(np.multiply(powers, x))
            if strength > best:
                best = strength
                best_x = x
                # print(strength, x)
    return best_x


d = dump(s, p, r)
d, np.sum(np.multiply(s, d)), np.sum(np.multiply(p.T, d), axis=1), r - np.sum(np.multiply(p.T, d), axis=1)

(array([80., 12.,  0.]),
 np.float64(380.0),
 array([448., 300., 404.]),
 array([ 2.,  0., 96.]))

In [31]:
@nb.njit('(float64[:], float64[:,:], float64[:])')
def test(s, p, r):
    for i in range(100):
        dump(s, p, r)


test(s, p, r)

Интересно, что процессор занят в 100%, но при этом потребление в ватах как в простое. Очевидно, что процессор вместо дела сидит и ждет когда ему освободят данные.

Исходя из этого предположения решил разделить данные для каждого потока

In [32]:
@nb.njit('float64(float64[:], float64[:,:], float64[:], float64[:], int64)', parallel=True)
def dump_new(powers, prices, resources, xs, depth):
    if len(powers) - depth > 1:
        best_x = -1
        best = np.zeros_like(powers)
        for x in range(0, int(np.min(
                np.divide(resources - np.sum(np.multiply(prices.T, xs), axis=1), prices[depth]))) + 1):
            new_xs = xs.copy()
            new_xs[depth] = x
            p = dump_new(powers, prices, resources, new_xs, depth + 1)
            if p > best_x:
                best_x = p
                best = new_xs
        xs[:] = best[:]
        return best_x
    else:
        x = int(np.floor(np.min(np.divide(resources - np.sum(np.multiply(prices.T, xs), axis=1), prices[-1]))))
        if x >= 0:
            xs[-1] = x
            return np.sum(np.multiply(powers, xs))
        else:
            return -1


xs = np.zeros_like(s).astype('float64')
x = dump_new(s, p, r, xs, 0)
d = xs
d, np.sum(np.multiply(s, d)), np.sum(np.multiply(p.T, d), axis=1), r - np.sum(np.multiply(p.T, d), axis=1)

(array([80., 12.,  0.]),
 np.float64(380.0),
 array([448., 300., 404.]),
 array([ 2.,  0., 96.]))

In [33]:
@nb.njit('(float64[:], float64[:,:], float64[:])')
def test(s, p, r):
    for i in range(100):
        dump_new(s, p, r, np.zeros_like(s).astype('float64'), 0)


test(s, p, r)

Как видно стало хуже, но теперь процессор работает на полную, упираясь в температурный лимит.

In [45]:
@nb.njit('float64(float64[:], float64[:,:], float64[:], float64[:], int64)')
def dump_new_one(powers, prices, resources, xs, depth):
    if len(powers) - depth > 1:
        best_x = -1
        best = np.zeros_like(powers)
        for x in range(0, int(np.min(
                np.divide(resources - np.sum(np.multiply(prices.T, xs), axis=1), prices[depth]))) + 1):
            new_xs = xs.copy()
            new_xs[depth] = x
            p = dump_new(powers, prices, resources, new_xs, depth + 1)
            if p > best_x:
                best_x = p
                best = new_xs
        xs[:] = best[:]
        return best_x
    else:
        x = int(np.floor(np.min(np.divide(resources - np.sum(np.multiply(prices.T, xs), axis=1), prices[-1]))))
        if x >= 0:
            xs[-1] = x
            return np.sum(np.multiply(powers, xs))
        else:
            return -1


xs = np.zeros_like(s).astype('float64')
x = dump_new_one(s, p, r, xs, 0)
d = xs
d, np.sum(np.multiply(s, d)), np.sum(np.multiply(p.T, d), axis=1), r - np.sum(np.multiply(p.T, d), axis=1)

(array([80., 12.,  0.]),
 np.float64(380.0),
 array([448., 300., 404.]),
 array([ 2.,  0., 96.]))

In [46]:
@nb.njit('(float64[:], float64[:,:], float64[:])')
def test(s, p, r):
    for i in range(100):
        dump_new_one(s, p, r, np.zeros_like(s).astype('float64'), 0)


test(s, p, r)

В один поток всё равно примерно в 24 раза быстрее (столько же сколько у меня ядер, совпадение ли?). Предполагаю, что это связано с тем, что даже при разделении памяти, всё равно присутствуют ожидания доступа на копирование этой памяти, которые в многопотоке не ускоришь.

In [47]:
@nb.njit('float64[:](float64[:], float64[:,:], float64[:])')
def dump(powers, prices, resources):
    best = 0
    best_x = np.zeros_like(powers)

    # maxes = np.floor(np.min(1 / np.divide(prices, resources.T), axis=1))
    maxes = np.floor(np.array([np.min(r) for r in 1 / np.divide(prices, resources.T)]))

    for X1 in range(0, int(maxes[0])):
        for X2 in range(0, int(maxes[1])):
            # print(resources - (prices[0] * X1 + prices[1] * X2), prices[2])
            x = np.array(
                [X1, X2, np.floor(np.min(np.divide(resources - (prices[0] * X1 + prices[1] * X2), prices[2])))])
            if np.any(x < 0):
                break
            strength = np.sum(np.multiply(powers, x))
            if strength > best:
                best = strength
                best_x = x
                # print(strength, x)
    return best_x


d = dump(s, p, r)
d, np.sum(np.multiply(s, d)), np.sum(np.multiply(p.T, d), axis=1), r - np.sum(np.multiply(p.T, d), axis=1)

(array([80., 12.,  0.]),
 np.float64(380.0),
 array([448., 300., 404.]),
 array([ 2.,  0., 96.]))

In [48]:
@nb.njit('(float64[:], float64[:,:], float64[:])')
def test(s, p, r):
    for i in range(100):
        dump(s, p, r)


test(s, p, r)

Версия без копирования памяти еще в 6 раз быстрее

Сравним с однопотоком питона

In [49]:
def dump(powers, prices, resources):
    best = 0
    best_x = np.zeros_like(powers)

    # maxes = np.floor(np.min(1 / np.divide(prices, resources.T), axis=1))
    maxes = np.floor(np.array([np.min(r) for r in 1 / np.divide(prices, resources.T)]))

    for X1 in range(0, int(maxes[0])):
        for X2 in range(0, int(maxes[1])):
            # print(resources - (prices[0] * X1 + prices[1] * X2), prices[2])
            x = np.array(
                [X1, X2, np.floor(np.min(np.divide(resources - (prices[0] * X1 + prices[1] * X2), prices[2])))])
            if np.any(x < 0):
                break
            strength = np.sum(np.multiply(powers, x))
            if strength > best:
                best = strength
                best_x = x
                # print(strength, x)
    return best_x


d = dump(s, p, r)
d, np.sum(np.multiply(s, d)), np.sum(np.multiply(p.T, d), axis=1), r - np.sum(np.multiply(p.T, d), axis=1)

(array([80., 12.,  0.]),
 np.float64(380.0),
 array([448., 300., 404.]),
 array([ 2.,  0., 96.]))

In [50]:
def test(s, p, r):
    for i in range(100):
        dump(s, p, r)


test(s, p, r)

В 16 раз медленнее

# Метод Гомори

In [40]:
def gomori_max(table: np.ndarray, columns: List[str] = None, rows: List[str] = None) -> dict:
    iteration = 0
    columns = ['free'] + [f'X{i}' for i in range(1, table.shape[1])] if columns is None else columns
    rows = [f'X{i + table.shape[1]}' for i in range(table.shape[0] - 1)] + ['F'] if rows is None else rows
    table = np.array(table).copy()
    table = table + Fraction()
    prev_leading_col = None
    r = np.vectorize(round)

    start_table = pd.DataFrame(table, columns=columns, index=rows)
    # max_iter = math.e ** np.sum(start_table.shape)

    print(iteration)
    print(start_table)

    def step(table: np.ndarray, columns: List[str], rows: List[str], leading_row=None, leading_col=None,
             optimization=True) -> dict:
        nonlocal iteration, prev_leading_col

        iteration += 1
        print("iteration:", iteration)
        # print(pd.DataFrame(table, columns=columns, index=rows))

        table2 = table.copy()
        if prev_leading_col is not None:
            table2[0, prev_leading_col] = 0

        if leading_col is None:
            leading_col = np.where(table2[-1, 1:] <= 0)[0][0] + 1 if optimization and sum(table2[-1, 1:] <= 0) == 1 \
                else np.abs(table2[:-1, 1:]).argmax() % (table.shape[1] - 1) + 1

        prev_leading_col = leading_col

        if leading_row is None:
            infs = np.logical_or(table2.T[leading_col, :-1] == 0,
                                 np.logical_and(table2.T[0, :-1] >= 0, table2.T[leading_col, :-1] <= 0))
            infs = np.append(infs, False)
            # print(infs)
            table2.T[0, infs] = np.inf
            table2.T[leading_col, infs] = 1
            # print(pd.DataFrame(table2.T, columns=rows, index= columns), leading_col, table2.T[0, :-1], table2.T[leading_col, :-1])
            col = np.divide(table2.T[0, :-1], table2.T[leading_col, :-1])
            leading_row = np.abs(col).argmin()

            # print("col:", col)

        table2[0] = table[leading_row] / table[leading_row, leading_col]
        table2[0, leading_col] = 1 / table[leading_row, leading_col]
        auxiliary = -table.T[leading_col]
        auxiliary = np.delete(auxiliary, leading_row)
        table = np.delete(table, leading_row, axis=0)
        table.T[leading_col] = 0

        for i in range(auxiliary.shape[0]):
            table2[i + 1] = table2[0] * auxiliary[i] + table[i]

        table = table2

        changed_col = columns[leading_col]
        columns[leading_col] = rows.pop(leading_row)
        rows.insert(0, changed_col)

        print("leading col, row:", leading_col, leading_row)
        print(pd.DataFrame(table, columns=columns, index=rows))
        print(auxiliary)

        return table

    while np.sum(table[-1] >= 0) < table.shape[1]:
        table = step(table, columns, rows)

    target_row = table[np.isin(rows, start_table.columns)]
    target_row = target_row[np.abs(target_row[:, 0] - r(target_row[:, 0])) != 0]

    while len(target_row) > 0 and iteration < 100:
        target_row = target_row - np.floor(target_row)
        target_row = target_row[target_row[:, 0].argmax()]
        rows.insert(len(rows) - 1, f'A{np.sum(table.shape) - np.sum(start_table.shape) + 1}')
        table = np.insert(table, [len(rows) - 2], -target_row, axis=0)
        print("iteration:", iteration + 0.5)
        print(pd.DataFrame(table, columns=columns, index=rows))
        target_row[target_row == 0] = np.nan
        table = step(table, columns, rows, leading_row=-2, leading_col=(target_row[0] / np.abs(target_row[1:])).argmin() + 1)
        # while np.sum(table[-1] >= 0) < table.shape[1] and iteration < 100:
        #     table = step(table, columns, rows)
        target_row = table[np.isin(rows, start_table.columns)]
        target_row = target_row[np.abs(target_row[:, 0] - r(target_row[:, 0])) != 0]

    print("iterations:", iteration)
    values = pd.DataFrame(table, columns=columns, index=rows).iloc[:, 0].round().astype(int).to_dict()
    for i in range(1, len(columns)):
        values[columns[i]] = 0
    return values


gomori_max(np.array([
    [6, 2, 1],
    [5, 1, 4],
    [0, -3, -2],
]))

0
   free  X1  X2
X3    6   2   1
X4    5   1   4
F     0  -3  -2
iteration: 1
leading col, row: 2 1
    free    X1    X4
X2   5/4   1/4   1/4
X3  19/4   7/4  -1/4
F    5/2  -5/2   1/2
[Fraction(-1, 1) Fraction(2, 1)]
iteration: 2
leading col, row: 1 1
    free    X3    X4
X1  19/7   4/7  -1/7
X2   4/7  -1/7   2/7
F   65/7  10/7   1/7
[Fraction(-1, 4) Fraction(5, 2)]
iteration: 2.5
    free    X3    X4
X1  19/7   4/7  -1/7
X2   4/7  -1/7   2/7
A1  -5/7  -4/7  -6/7
F   65/7  10/7   1/7
iteration: 3
leading col, row: 2 -2
    free    X3    A1
X4   5/6   2/3  -7/6
X1  17/6   2/3  -1/6
X2   1/3  -1/3   1/3
F   55/6   4/3   1/6
[Fraction(1, 7) Fraction(-2, 7) Fraction(-1, 7)]
iteration: 3.5
    free    X3    A1
X4   5/6   2/3  -7/6
X1  17/6   2/3  -1/6
X2   1/3  -1/3   1/3
A2  -5/6  -2/3  -5/6
F   55/6   4/3   1/6
iteration: 4
leading col, row: 2 -2
   free    X3    A2
A1    1   4/5  -6/5
X4    2   8/5  -7/5
X1    3   4/5  -1/5
X2    0  -3/5   2/5
F     9   6/5   1/5
[Fraction(7, 6) Fractio

{'A1': 1, 'X4': 2, 'X1': 3, 'X2': 0, 'F': 9, 'X3': 0, 'A2': 0}

In [41]:
Xs = [[Rv] + [Uv[Rk] for Uk, Uv in U.items()] for Rk, Rv in R.items()] + [[0] + [-Uk[0] for Uk, Uv in U.items()]]
np.array(Xs)

array([[450,   5,   4,  50],
       [300,   3,   5,  35],
       [500,   4,   7,  45],
       [  0,  -4,  -5, -40]])

In [42]:
j = gomori_max(Xs, rows=['Z', 'D', 'K', 'F'], columns=['free', 'X1', 'X2', 'X3'])
j = {i: j[i] for i in ['Z', 'D', 'K', 'F'] + ['X1', 'X2', 'X3'] + ['F']}
j

0
  free  X1  X2   X3
Z  450   5   4   50
D  300   3   5   35
K  500   4   7   45
F    0  -4  -5  -40
iteration: 1
leading col, row: 3 1
      free    X1     X2      D
X3    60/7  3/35    1/7   1/35
Z    150/7   5/7  -22/7  -10/7
K    800/7   1/7    4/7   -9/7
F   2400/7  -4/7    5/7    8/7
[Fraction(-50, 1) Fraction(-45, 1) Fraction(40, 1)]
iteration: 2
leading col, row: 1 1
   free      Z     X2    D
X1   30    7/5  -22/5   -2
X3    6  -3/25  13/25  1/5
K   110   -1/5    6/5   -1
F   360    4/5   -9/5    0
[Fraction(-3, 35) Fraction(-1, 7) Fraction(4, 7)]
iteration: 3
leading col, row: 2 1
       free      Z      X3       D
X2   150/13  -3/13   25/13    5/13
X1  1050/13   5/13  110/13   -4/13
K   1250/13   1/13  -30/13  -19/13
F   4950/13   5/13   45/13    9/13
[Fraction(22, 5) Fraction(-6, 5) Fraction(9, 5)]
iteration: 3.5
       free      Z      X3       D
X2   150/13  -3/13   25/13    5/13
X1  1050/13   5/13  110/13   -4/13
K   1250/13   1/13  -30/13  -19/13
A1   -10/13  -5/13   -

{'Z': 2, 'D': 0, 'K': 96, 'F': 380, 'X1': 80, 'X2': 12, 'X3': 0}

In [51]:
def test():
    for i in range(100):
        gomori_max(Xs, rows=['Z', 'D', 'K', 'F'], columns=['free', 'X1', 'X2', 'X3'])


test()

0
  free  X1  X2   X3
Z  450   5   4   50
D  300   3   5   35
K  500   4   7   45
F    0  -4  -5  -40
iteration: 1
leading col, row: 3 1
      free    X1     X2      D
X3    60/7  3/35    1/7   1/35
Z    150/7   5/7  -22/7  -10/7
K    800/7   1/7    4/7   -9/7
F   2400/7  -4/7    5/7    8/7
[Fraction(-50, 1) Fraction(-45, 1) Fraction(40, 1)]
iteration: 2
leading col, row: 1 1
   free      Z     X2    D
X1   30    7/5  -22/5   -2
X3    6  -3/25  13/25  1/5
K   110   -1/5    6/5   -1
F   360    4/5   -9/5    0
[Fraction(-3, 35) Fraction(-1, 7) Fraction(4, 7)]
iteration: 3
leading col, row: 2 1
       free      Z      X3       D
X2   150/13  -3/13   25/13    5/13
X1  1050/13   5/13  110/13   -4/13
K   1250/13   1/13  -30/13  -19/13
F   4950/13   5/13   45/13    9/13
[Fraction(22, 5) Fraction(-6, 5) Fraction(9, 5)]
iteration: 3.5
       free      Z      X3       D
X2   150/13  -3/13   25/13    5/13
X1  1050/13   5/13  110/13   -4/13
K   1250/13   1/13  -30/13  -19/13
A1   -10/13  -5/13   -

В 2-3 раза быстрее